In [1]:
import pandas as pd
import fiona
import folium
import geopandas as gpd
from folium import Choropleth, Circle, Marker
from folium.plugins import HeatMap, MarkerCluster
import math

In [2]:
Obesity_df = pd.read_csv("../data/3. Obesity-prevalence-by-state-2021.csv")
Diabeties_df = pd.read_csv("../data/US-CDI_diabetes_prevalence.csv")
df_fastfood = pd.read_csv("../data/fast_food.csv")
df_Pop = pd.read_csv("../data/Pop_dataUS.csv")

In [3]:
Obesity_df.head()

,State,,Prevalence,95% CI
0,Alabama,AL,39.9,"(37.9, 41.8)"
1,Alaska,AK,33.5,"(31.5, 35.5)"
2,Arizona,AZ,31.3,"(30.0, 32.6)"
3,Arkansas,AR,38.7,"(36.6, 40.8)"
4,California,CA,27.6,"(26.1, 29.1)"


In [4]:
Diabeties_df.head()

,Unnamed: 0,yearstart,locationabbr,locationdesc,question,datavalueunit,datavaluetype,datavalue
0,5841,2014,AK,Alaska,Prevalence of diagnosed diabetes among adults ...,%,Age-adjusted Prevalence,7.7
1,5927,2015,AK,Alaska,Prevalence of diagnosed diabetes among adults ...,%,Age-adjusted Prevalence,7.8
2,5869,2016,AK,Alaska,Prevalence of diagnosed diabetes among adults ...,%,Age-adjusted Prevalence,7.8
3,5926,2017,AK,Alaska,Prevalence of diagnosed diabetes among adults ...,%,Age-adjusted Prevalence,7.6
4,5856,2018,AK,Alaska,Prevalence of diagnosed diabetes among adults ...,%,Age-adjusted Prevalence,8.7


In [5]:
df_fastfood.head()

,address,latitude,longitude,name,province
0,800 N Canal Blvd,29.814697,-90.814742,SONIC Drive In,LA
1,800 N Canal Blvd,29.814697,-90.814742,SONIC Drive In,LA
2,206 Wears Valley Rd,35.803788,-83.580553,Taco Bell,TN
3,3652 Parkway,35.782339,-83.551408,Arby's,TN
4,2118 Mt Zion Parkway,33.562738,-84.321143,Steak 'n Shake,GA


In [6]:
df_Pop.head()

,SUMLEV,REGION,DIVISION,STATE,State_abbreviations,NAME,ESTIMATESBASE2020,POPESTIMATE2020,POPESTIMATE2021,NPOPCHG_2020,...,NETMIG2020,NETMIG2021,RESIDUAL2020,RESIDUAL2021,RBIRTH2021,RDEATH2021,RNATURALINC2021,RINTERNATIONALMIG2021,RDOMESTICMIG2021,RNETMIG2021
0,40,3,6,1,AL,Alabama,5024279,5024803,5039877,524,...,3337,23380,-75,242,11.191613,12.890226,-1.698613,0.247201,4.398749,4.645950
1,40,4,9,2,AK,Alaska,733391,732441,732673,-950,...,-2012,-3353,39,-54,12.667956,7.700425,4.967532,0.718033,-5.295151,-4.577118
2,40,4,8,4,AZ,Arizona,7151502,7177986,7276316,26484,...,26791,97504,-3,-6,10.584669,10.469547,0.115121,0.619608,12.871739,13.491347
3,40,3,7,5,AR,Arkansas,3011524,3012232,3025891,708,...,1618,16840,-15,55,11.599962,12.671819,-1.071856,0.272932,5.304960,5.577892
4,40,4,9,6,CA,California,39538223,39499738,39237836,-38485,...,-62562,-352960,858,-938,10.778412,8.441637,2.336775,0.364223,-9.329701,-8.965478


In [7]:
df_pop2 = df_Pop.iloc[:, [4,7]]
df_pop2.head()

,State_abbreviations,POPESTIMATE2020
0,AL,5024803
1,AK,732441
2,AZ,7177986
3,AR,3012232
4,CA,39499738


In [8]:
#geoJson File with US state boundries
url = ("https://raw.githubusercontent.com/python-visualization/folium/master/examples/data")
state_geo = f"{url}/us-states.json"

# We read the file and print it.
geoJSON_df = gpd.read_file(state_geo)
geoJSON_df = geoJSON_df.rename(columns={'id': 'province'})
geoJSON_df = geoJSON_df.set_index("province")
geoJSON_df.head()

,name,geometry
province,,
AL,Alabama,"POLYGON ((-87.35930 35.00118, -85.60667 34.984..."
AK,Alaska,"MULTIPOLYGON (((-131.60202 55.11798, -131.5691..."
AZ,Arizona,"POLYGON ((-109.04250 37.00026, -109.04798 31.3..."
AR,Arkansas,"POLYGON ((-94.47384 36.50186, -90.15254 36.496..."
CA,California,"POLYGON ((-123.23326 42.00619, -122.37885 42.0..."


In [9]:
#creating the base map
m = folium.Map(location=[40,-94], tiles='cartodbpositron', zoom_start=4)

#creating layer with population estimates on map
# Add a choropleth map to the base map
folium.Choropleth(
    geo_data=geoJSON_df.__geo_interface__, 
    data=df_pop2,
    columns=['State_abbreviations','POPESTIMATE2020'],
    key_on='feature.id',
    fill_color="YlGn",
    fill_opacity=0.7,
    line_opacity=.1,
    legend_name="State Population",
).add_to(m)